In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
sub = pd.read_csv("../input/gender_submission.csv")
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
test.info()

In [ ]:
train.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='inferno')

For filling null values in categorical column with a particular mode

In [ ]:
#train.loc[train['Pclass']==3, 'Cabin'] = train.loc[train['Pclass']==3, 'Cabin'].fillna(value=train.loc[train['Pclass']==3, 'Cabin'].mode().iloc[0])

In [ ]:
#sns.countplot(train['Survived'],hue=train['Sex'])

In [ ]:
#sns.pairplot(train,palette='coolwarm',hue='Sex')

In [ ]:
train = train[['Name','Sex','Cabin','Embarked','Ticket','Pclass','Age','SibSp','Parch','Fare','Survived']]
test = test[['Name','Sex','Cabin','Embarked','Ticket','Pclass','Age','SibSp','Parch','Fare']]

In [ ]:
train.head()

In [ ]:
def get_initials(name):
    point = name.index('.')
    res = ''
    for i in range(point-1,-1,-1):
        if name[i]==' ':
            break
        else:
            res+=name[i]
    res = res[len(res)-1::-1]
    return res

In [ ]:
train['Name'] = train['Name'].apply(lambda x: get_initials(x))
test['Name'] = test['Name'].apply(lambda x: get_initials(x))

In [ ]:
train['Cabin'].fillna(value='None',inplace=True)
test['Cabin'].fillna(value='None',inplace=True)

In [ ]:
train['Embarked'].fillna(value='None',inplace=True)
test['Embarked'].fillna(value='None',inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
def encode_features(train,test):
    features = ['Name','Sex','Cabin','Embarked','Ticket']
    comb = pd.concat([train[features],test[features]])
    
    for feature in features:
        le = LabelEncoder()
        le = le.fit(comb[feature])
        train[feature] = le.transform(train[feature])
        test[feature] = le.transform(test[feature])
    return train,test
train,test = encode_features(train,test)

In [ ]:
features = ['Name','Sex','Cabin','Embarked','Ticket']
train = pd.get_dummies(train,columns=features,drop_first=True)
test = pd.get_dummies(test,columns=features,drop_first=True)
missing_cols = set( train.columns ) - set( test.columns )
for c in missing_cols:
    test[c] = 0
test = test[train.columns]

In [ ]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values=np.nan,strategy='mean',axis=0)
imputer = imputer.fit(train.iloc[:,:5])
train.iloc[:,:5] = imputer.transform(train.iloc[:,:5])
test.iloc[:,:5] = imputer.transform(test.iloc[:,:5])

In [ ]:
print(test.info())
print()
print(train.info())

In [ ]:
y_train = train['Survived'].values
x_train = train.drop('Survived',axis=1).values
test = test.drop('Survived',axis=1).values
#x_train = np.concatenate((a_train,b_train),axis=1)
#test = np.concatenate((a_test,b_test),axis=1)

In [ ]:
"""from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
sc = sc.fit(x_train)
x_train = sc.transform(x_train)
test = sc.transform(test)"""

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=0.1)

ML Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=300)

#from xgboost import XGBClassifier
#model = XGBClassifier()

#from sklearn.svm import SVC
#model = SVC(kernel='linear')

#from sklearn.linear_model import LogisticRegression
#model = LogisticRegression()

model.fit(x_train,y_train)

In [ ]:
"""from keras.models import Sequential
from keras.layers import Dense,Dropout

model = Sequential()

model.add(Dense(output_dim=10,init='uniform',activation='relu',input_dim=852))
model.add(Dropout(0.2))
model.add(Dense(output_dim=10,init='uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,batch_size=20,epochs=15)"""

In [ ]:
"""from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def build(optimizer):
    model = Sequential()
    model.add(Dense(output_dim=10,init='uniform',activation='relu',input_dim=852))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim=10,init='uniform',activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=build)

parameters = [{'batch_size':[10,20,30] ,
              'epochs':[5,10,20,15] , 
              'optimizer':['adam']}]

grid_search = GridSearchCV(estimator=model,param_grid=parameters,scoring='accuracy',cv=10)

grid_search = grid_search.fit(x_train,y_train)"""


In [ ]:
#best_params = grid_search.best_params_
#print(best_params)
#print(grid_search.best_score_)

In [ ]:
pred = model.predict(x_test)
#pred = grid_search.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
pred = pred>0.5
print(classification_report(y_test,pred))
print()
print(confusion_matrix(y_test,pred))

In [ ]:
predictions = model.predict(test)
#predictions = grid_search.predict(test)

In [ ]:
submission = pd.read_csv('../input/test.csv')
final = submission[['PassengerId']].copy()
final['Survived'] = predictions
del submission

In [ ]:
final['Survived'] = final['Survived'] > 0.5

In [ ]:
def nums(value):
    if value==False:
        value = '0'
    else:
        value = '1'
    return value
final['Survived'] = final['Survived'].apply(lambda x :nums(x))

In [ ]:
final.to_csv('titanic_pred.csv',index=False)